###### create a environment as we normally create a virtual environment; and install all dependencies in that ^ **ENVIRONMENT**


In [ ]:
try:
    import reverse_geocoder
    print("reverse_geocoder is installed.")
except ImportError:
    print("reverse_geocoder is not installed.")

# %pip install reverse_geocoder

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 33, Finished, Available, Finished)

reverse_geocoder is installed.


In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
# to extract the country code, make sure it is installed in your fabric <pip install reverse_geocoder>
import reverse_geocoder as rg

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 34, Finished, Available, Finished)

### **Read the data from silver layer**

In [ ]:
df = spark.read.table("earthquakes_lakehouse.earthquake_events_silver")

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 35, Finished, Available, Finished)

### **UDF Experiments**

In [ ]:
def get_country_code(lat, lon):
    try:
        coordinates = (float(lat), float(lon))
        # return Country code of the location
        return rg.search(coordinates)[0].get('cc'), rg.search(coordinates)[0].get('name') # Country Code and Name
    except Exception as e:
        return None, None

# test case to retrieve country code
print(get_country_code(11.0588, -86.9377))

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 36, Finished, Available, Finished)

('NI', 'Masachapa')


### **Register the function as UDF**

In [ ]:
# Define UDF schema
geo_schema = StructType([
    StructField("country_code", StringType(), True),
    StructField("country_name", StringType(), True)
])


get_country_info_udf = udf(get_country_code, geo_schema)

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 37, Finished, Available, Finished)

In [ ]:
df_with_location = df.withColumn('country_code', get_country_info_udf(col("latitude"), col("longitude"))['country_code'])\
                     .withColumn('country_name', get_country_info_udf(col("latitude"), col("longitude"))['country_name'])\
                     .drop("magType_joined")


display(df_with_location)
# df_with_location.printSchema()

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 38, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f3dc0f14-24ed-49bf-bfa1-70b3742e96af)

#### **Time to move everything to gold and apply merge by using delta table**

In [ ]:
from delta.tables import DeltaTable

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 39, Finished, Available, Finished)

In [ ]:
if not spark.catalog.tableExists('earthquakes_lakehouse.earthquake_events_gold'):
    print("Table Doest Exists")
    df_with_location.write.mode('overwrite').format('delta').option("mergeSchema", "true").saveAsTable("earthquakes_lakehouse.earthquake_events_gold")
else:
    gold_delta_tbl = DeltaTable.forName(
        spark, 
        "earthquakes_lakehouse.earthquake_events_gold"
    )


    gold_delta_tbl.alias("gold")\
        .merge(
            df_with_location.alias("updates"),
            "gold.id = updates.id"
        ).whenMatchedUpdate(
            set={

            }
        ).whenNotMatchedInsert(
            values={
                "id": "updates.id",
                "longitude": "updates.longitude",
                "latitude": "updates.latitude",
                "elevation": "updates.elevation",
                "title": "updates.title",
                "place_description": "updates.place_description",
                "type": "updates.type",
                "magnitude": "updates.magnitude",
                "magType": "updates.magType",
                "time": "updates.time",
                "timestamp": "updates.timestamp",
                "country_code": "updates.country_code",
                "country_name": "updates.country_name"
            }
        ).execute()

StatementMeta(, 24cbd03d-a1b7-4d85-861d-c3c5395afc23, 40, Finished, Available, Finished)